# Introduction to Data Science Project Code

## By Jack Mittenthal, Aaron Niecestro, Yukuan Pan, Jung Yang

<b> Please read the following message below before running the code <b>

<b> WARNING: Running the code below will take between 2-3 hours. <b>

### Packages Imported 

In [5]:
from Bio import Entrez 
import time
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import string
from tabulate import tabulate
import sqlite3

### Question 1

#### Part I) A scraper module that can collect paper title, author list, publication time, and abstract from PUBMED for the keyword “HIV” within a pre-specified time window 01/01/2020 – 08/30/2020, and the retrieveddata should be saved in the CSV format.

In [2]:
# Function that scapes the data from Pubmed, and loads it into a csv file
def Q1():
    # Specify the keyword
    keyword = "HIV"
    # The time window that we are looking for 
    startdate = "2020/01/01"
    enddate = "2020/08/30"
    
    # Retrieve the search results from PUBMED using BioPython
    Entrez.email = "aaron.niecestro@uth.tmc.edu"  # Always tell NCBI who you are before moving on
    handle = Entrez.esearch(db="pubmed", term=keyword, mindate=startdate, maxdate=enddate, retmax = 10000)
    ids = Entrez.read(handle)["IdList"]
    
    # I usually create a list of dictionaries before converting to a dataframe then csv at the end
    data = []
    for id_ in tqdm(ids):
        try: # if this code fails just continue
            a = Entrez.read(Entrez.efetch(db="pubmed", id=id_))
        except:
            continue
        
        try:
            title = a["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleTitle"]
        except:
            continue
        
        author_list = ""
        try:
            for x in a["PubmedArticle"][0]["MedlineCitation"]["Article"]["AuthorList"]:
                #Note: Sometimes it is a collective and the names are not found
                if ("CollectiveName" in x):
                    author_list += x["CollectiveName"]
                elif ("ForeName" in x):
                    author_list += x["ForeName"] + " " +  x["LastName"]
                #Weird case I found
                else:
                    author_list = x["LastName"]
                
                if (x != a["PubmedArticle"][0]["MedlineCitation"]["Article"]["AuthorList"][-1]):
                    author_list += ", "
        except:
            continue

        #Note: sometimes a date of publication isn't given...
        # need only 01/01/2020 to 08/31/200, not before or after
        try:
            pub_time_dict = a["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleDate"][0]
            pub_time = pub_time_dict["Year"] + "/" + pub_time_dict["Month"] + "/" + pub_time_dict["Day"]
        except:
            continue

        #if we do a big abstract of just text
        abstract = ""
        try:
            for i in range(len(a["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"])):
                abstract += a["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][i]
        #Note: sometimes abstract is missing
        except KeyError:
            continue

        data.append({"title":title,"author_list":author_list,"pub_time":pub_time,"abstract":abstract})

    df = pd.DataFrame(data)
    df.to_csv("pubmed_capstone.csv", index=False)

In [3]:
Q1() # takes about 2 hours to load completely if you leave it in the background. More time used if minimized.

100%|██████████| 9999/9999 [2:00:22<00:00,  1.38it/s]  


#### Part II) Cleaning the dataset

In [13]:
df = pd.read_csv('C:\\Users\\aniec\\Documents\\pubmed_capstone.csv')

In [14]:
# Check the dimensions of the dataframe
df.shape

(6622, 4)

In [15]:
# Extract Year from pub_time column
df['year'] = pd.DatetimeIndex(df['pub_time']).year

# Extract Month from pub_time column
df['month'] = pd.DatetimeIndex(df['pub_time']).month

# Convert Month Number to Month Name
df['month_name'] = pd.DatetimeIndex(df['pub_time']).month_name().str[:3]

# Extract Day from pub_time column
df['day'] = pd.DatetimeIndex(df['pub_time']).day

# Change title column into a string 
df['title'] = df['title'].astype("string")

# Change author_list column into a string
df['author_list'] = df['author_list'].astype("string")

# Change abstract column into a string
df['abstract'] = df['abstract'].astype("string")

# Change abstract column into a string
df['month_name'] = df['month_name'].astype("string")

# Convert pub_time string type column into a datetime type column
df[['pub_time']] = df[['pub_time']].apply(pd.to_datetime)

In [16]:
# Fixing the dataframe to show only correct dates
df2 = df[(df['pub_time'] >= '2020-01-01') & (df['pub_time'] <= '2020-08-30')]

In [17]:
# WebScapping Goal: Contains HIV in the abstract
# Would use title too but felt it was more relevant with abstract only
df3 = df2[df2['abstract'].str.contains("HIV")]

In [18]:
# Sort the Article Publication Time observations from oldest to newest
df3.sort_values(by=['pub_time'])

,title,author_list,pub_time,abstract,year,month,month_name,day
6334,Association between Knowledge of Sexually Tran...,"Imran O Morhason-Bello, Adeniyi F Fagbamigbe",2020-01-02,<i>Background.</i> Adequate knowledge of sexua...,2020,1,Jan,2
6053,Scrofula Caused by Multidrug-Resistant Tubercu...,"Bárbara Pedro, Liliana Alves, Rita Magano, Tom...",2020-01-02,Tuberculosis (TB) is one of the top 10 causes ...,2020,1,Jan,2
6496,A Chimeric Sudan Virus-Like Particle Vaccine C...,"Fangfang Wu, Shengnan Zhang, Ying Zhang, Ruo M...",2020-01-03,Ebola virus infections lead to severe hemorrha...,2020,1,Jan,3
6267,Rapidly progressive necrotizing cellulitis sec...,"Nithya Krishnan, Bijal Patel, William Palfrey,...",2020-01-03,The incidence of invasive fungal infection is ...,2020,1,Jan,3
4081,Restriction of exogenous DNA expression by SAM...,"Chenjian Gu, Lijun Ming, Yili Fang, Xuejing Li...",2020-01-03,SAMHD1 (Sterile Alpha Motif and Histidine-aspa...,2020,1,Jan,3
...,...,...,...,...,...,...,...,...
15,Social determinants of mental health care syst...,"Ish P Bhalla, Elina A Stefanovics, Robert A Ro...",2020-08-28,Since deinstitutionalization in the 1950s-1970...,2020,8,Aug,28
47,The burden of laboratory-confirmed pertussis i...,"Rudzani Muloiwa, Benjamin M Kagina, Mark E Eng...",2020-08-28,An effective vaccine against Bordetella pertus...,2020,8,Aug,28
7,Comorbidities and HCV coinfection in the manag...,"Elisabetta Garagiola, Emanuela Foglia, Lucrezi...",2020-08-29,Since HIV+ treatment has become more effective...,2020,8,Aug,29
6,Development of a composite indicator to priori...,"Shrikala Acharya, Maninder Singh Setia, Amol P...",2020-08-29,A key recommendation of the National AIDS Cont...,2020,8,Aug,29


In [19]:
# Show each column type in the dataframe
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5540 entries, 0 to 6619
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   title        5540 non-null   string        
 1   author_list  5540 non-null   string        
 2   pub_time     5540 non-null   datetime64[ns]
 3   abstract     5540 non-null   string        
 4   year         5540 non-null   int32         
 5   month        5540 non-null   int32         
 6   month_name   5540 non-null   string        
 7   day          5540 non-null   int32         
dtypes: datetime64[ns](1), int32(3), string(4)
memory usage: 324.6 KB


In [20]:
# Show dimensions of the dataframe
df3.shape

(5540, 8)

In [21]:
# Show first 5 observations of the dataframe
df3.head()

,title,author_list,pub_time,abstract,year,month,month_name,day
0,High microbial translocation limits gut immune...,"Doungnapa Kantamala, Jutarat Praparattanapan, ...",2020-08-26,Individuals residing in areas with high preval...,2020,8,Aug,26
1,High sleep-related breathing disorders among H...,"Chang-Chun Chen, Cheng-Yu Lin, Yen-Chin Chen, ...",2020-07-20,Sleep-related breathing disorders (SRBD) not o...,2020,7,Jul,20
2,"Design, synthesis and SAR study of novel C2-py...","Manoj Patel, Christopher Cianci, Christopher W...",2020-08-27,"The design, synthesis and structure-activity r...",2020,8,Aug,27
4,Association of maternal and infant inflammatio...,"Tatum Sevenoaks, Catherine J Wedderburn, Kirst...",2020-08-26,HIV-exposed uninfected (HEU) children may have...,2020,8,Aug,26
5,Head-to-head comparison of Enzyme Linked Immun...,"Aseem Kumar Tiwari, Anand Prakash Upadhyay, Di...",2020-08-26,Safe blood transfusion being the cornerstone o...,2020,8,Aug,26


In [22]:
# Checking Missing observations
df3.isnull().sum()
# Missing data = None, so we do not have to get rid of anything

title          0
author_list    0
pub_time       0
abstract       0
year           0
month          0
month_name     0
day            0
dtype: int64

In [23]:
# Convert dataframe into a new csv file
df3.to_csv("pubmed_capstone2.csv", index=False)

### Question 2) A database module that can import the CSV file to SQLite to build a database automatically. Then implement SQL code to query the publications by author’s name (i.e., input an author’s name and find out and return all his/her publications).

In [24]:
# Loading the CSV file as a dataframe called PubmedCsv
PubmedCsv = pd.read_csv('C:\\Users\\aniec\\Documents\\pubmed_capstone2.csv')

In [25]:
# Connecting SQL lite and creating a database called pubmed.db
conn = sqlite3.connect("pubmed.db")

In [26]:
# Converting the dataframe to a SQL table. SQL table is called pubmedTable
PubmedCsv.to_sql("pubmedTable", conn, if_exists= 'replace', index= False)

5540

In [27]:
# Reading in a SQL query from python
pubmedSql = pd.read_sql_query('Select * from pubmedTable limit 5;', conn)

In [28]:
# Loading the query
pubmedSql

,title,author_list,pub_time,abstract,year,month,month_name,day
0,High microbial translocation limits gut immune...,"Doungnapa Kantamala, Jutarat Praparattanapan, ...",2020-08-26,Individuals residing in areas with high preval...,2020,8,Aug,26
1,High sleep-related breathing disorders among H...,"Chang-Chun Chen, Cheng-Yu Lin, Yen-Chin Chen, ...",2020-07-20,Sleep-related breathing disorders (SRBD) not o...,2020,7,Jul,20
2,"Design, synthesis and SAR study of novel C2-py...","Manoj Patel, Christopher Cianci, Christopher W...",2020-08-27,"The design, synthesis and structure-activity r...",2020,8,Aug,27
3,Association of maternal and infant inflammatio...,"Tatum Sevenoaks, Catherine J Wedderburn, Kirst...",2020-08-26,HIV-exposed uninfected (HEU) children may have...,2020,8,Aug,26
4,Head-to-head comparison of Enzyme Linked Immun...,"Aseem Kumar Tiwari, Anand Prakash Upadhyay, Di...",2020-08-26,Safe blood transfusion being the cornerstone o...,2020,8,Aug,26


In [29]:
# let user input user name
authorName = input("please enter the author name:")

please enter the author name:Doungnapa Kantamala


In [30]:
# Reading the SQL authorName Query 
results = pd.read_sql_query("select title from pubmedTable where author_list like '%{}%';".format(authorName), conn)

In [31]:
# Load the SQL created above for Authors Name
results

,title
0,High microbial translocation limits gut immune...


In [33]:
# Closing the connection to SQL
conn.close()

### Question 3) A visualization module

#### Part I) Read the csv file

In [34]:
# Converting the csv file to dataframe for the visualations
df = pd.read_csv('C:\\Users\\aniec\\Documents\\pubmed_capstone2.csv')
df.head()

,title,author_list,pub_time,abstract,year,month,month_name,day
0,High microbial translocation limits gut immune...,"Doungnapa Kantamala, Jutarat Praparattanapan, ...",2020-08-26,Individuals residing in areas with high preval...,2020,8,Aug,26
1,High sleep-related breathing disorders among H...,"Chang-Chun Chen, Cheng-Yu Lin, Yen-Chin Chen, ...",2020-07-20,Sleep-related breathing disorders (SRBD) not o...,2020,7,Jul,20
2,"Design, synthesis and SAR study of novel C2-py...","Manoj Patel, Christopher Cianci, Christopher W...",2020-08-27,"The design, synthesis and structure-activity r...",2020,8,Aug,27
3,Association of maternal and infant inflammatio...,"Tatum Sevenoaks, Catherine J Wedderburn, Kirst...",2020-08-26,HIV-exposed uninfected (HEU) children may have...,2020,8,Aug,26
4,Head-to-head comparison of Enzyme Linked Immun...,"Aseem Kumar Tiwari, Anand Prakash Upadhyay, Di...",2020-08-26,Safe blood transfusion being the cornerstone o...,2020,8,Aug,26


#### Part II) Show the number of publications in each month

In [44]:
fig = px.bar(df.groupby(['month_name'], as_index=False)['title'].count(), 
             x='month_name', y='title', title="Publications per Month",
             labels={"month_name": "Month",
                     "title": "Number of Publications",
                    },
             category_orders={ # replaces default order by column name
                 "month_name": ["Jan", "Feb", "Mar", 'Apr', "May", "Jun", 'Jul', 'Aug'],
             },
             template="plotly_white"
              )

fig.update_layout(
    title={'text': '<b>Publications per Month in 2022<b>',
           'y':0.9,
           'x':0.5,
           'font': {'size': 20, 'color': 'black'},
           'xanchor': 'center',
           'yanchor': 'top'})

fig.update_xaxes(
    title={'text': 'Month',
           'font': {'size': 15, 'color': 'black'},})

fig.update_yaxes(
    title={'text': 'Number of Publications',
           'font': {'size': 15, 'color': 'black'},})

fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.update_yaxes(range=[0, 900])

fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=679.625, y1=679.625, yref="y")

fig.add_annotation( # add a text callout with arrow
    text="Lowest Month!", x="Jan", y=550, arrowhead=1, showarrow=True )

fig.add_annotation( # add a text callout with arrow
    text="Above Average!", x="May", y=750, arrowhead=1, showarrow=True )

fig.add_annotation( # add a text callout with arrow
    text="Highest Month!", x="Jul", y=790, arrowhead=1, showarrow=True )

fig.add_annotation( # add a text callout with arrow
    text="Below Average!", x="Aug", y=640, arrowhead=1, showarrow=True)

fig.update_traces(marker_color='black')

fig.show()

#### Part III) Generate and visualize the summary statistics for the publication numbers per month, including mean, SD, range, median, 1st to 3rd quartile

In [36]:
# Creating Series that show the Summary Statistics
series_avg = pd.Series(df.groupby(['month'], as_index=False)['title'].count().mean())
series_min = pd.Series(df.groupby(['month'], as_index=False)['title'].count().min())
series_max = pd.Series(df.groupby(['month'], as_index=False)['title'].count().max())
series_med = pd.Series(df.groupby(['month'], as_index=False)['title'].count().median())
series_std = pd.Series(df.groupby(['month'], as_index=False)['title'].count().std())
series_25 = pd.Series(df.groupby(['month'], as_index=False)['title'].count().quantile(0.25)) 
series_75 = pd.Series(df.groupby(['month'], as_index=False)['title'].count().quantile(0.75)) 
series_sum = pd.Series(df.groupby(['month'], as_index=False)['title'].count().sum())

In [37]:
# Converting the series into dataframe that show the Summary Statistics 
df_avg = series_avg.to_frame()
df_sum = series_sum.to_frame()
df_min = series_min.to_frame()
df_max = series_max.to_frame()
df_med = series_med.to_frame()
df_std = series_std.to_frame()
df_25 = series_25.to_frame()
df_75 = series_75.to_frame()

In [38]:
# Combining the dataframes into one to show the Summary Statistics
result = pd.concat([df_sum, df_avg, df_min, df_max, df_std, df_25, df_med, df_75], axis=1, join='inner')
result.columns = ['Total', 'Mean', 'Min', 'Max', 'Standard Deviation', 'IQR 25%', 'Median', 'IQR 75%']

In [39]:
# Drop the first row in the dataframe since it is not needed
result2 = result.iloc[1:, 0:]

In [40]:
# Printing the Dataframe
print(tabulate(result2, headers = 'keys', tablefmt = 'simple'))

         Total    Mean    Min    Max    Standard Deviation    IQR 25%    Median    IQR 75%
-----  -------  ------  -----  -----  --------------------  ---------  --------  ---------
title     5540   692.5    529    770               81.5248        656     734.5        738


#### Part IV) Visualize the trend of the publication numbers over time (by months)

In [41]:
# Have to reorder the months to work in correct order before making plot like below
df2 = df
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug"]
df2['month_name'] = pd.Categorical(df2['month_name'], categories=months, ordered=True)
df.sort_values(by='month_name',inplace=True)

In [42]:
fig = px.line(df.groupby(['month_name'], as_index=False)['title'].count(), 
              x='month_name', y='title', title="Publications per Month",
              labels={"month_name": "Month",
                     "title": "Number of Publications",
                     },
              category_orders={ # replaces default order by column name
                 "month_name": ["Jan", "Feb", "Mar", 'Apr', "May", "Jun", 'Jul', 'Aug'],
              },
              template="plotly_white"
             )

fig.update_layout(
    title={'text': '<b>Publications per Month in 2022<b>',
           'y':0.9,
           'x':0.5,
           'font': {'size': 20, 'color': 'black'},
           'xanchor': 'center',
           'yanchor': 'top'})

fig.update_xaxes(
    title={'text': 'Month',
           'font': {'size': 15, 'color': 'black'},})

fig.update_yaxes(
    title={'text': 'Number of Publications',
           'font': {'size': 15, 'color': 'black'},})

fig.add_shape( # add a horizontal "target" line
    type="line", line_color="blue", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=679.625, y1=679.625, yref="y")

fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.update_xaxes(range=[-0.1, 7.1])
fig.update_yaxes(range=[0, 800])
fig.update_traces(line_color='black', mode='lines+markers')

fig.show()

In [43]:
fig = px.line(df.groupby(['pub_time'], as_index=False)['title'].count(), 
              x='pub_time', 
              y='title', 
              title="Publications per Month",
              labels={"pub_time": "Publication day",
                      "title": "Number of Publications",
                 },
              template="plotly_white"
              )

fig.update_layout(
    title={'text': '<b>Publications per Day Each Month<b>',
           'y':0.9,
           'x':0.5,
           'font': {'size': 20, 'color': 'black'},
           'xanchor': 'center',
           'yanchor': 'top'})

fig.update_xaxes(
    title={'text': 'Publication Month',
           'font': {'size': 15, 'color': 'black'},})

fig.update_yaxes(
    title={'text': 'Number of Publications',
           'font': {'size': 15, 'color': 'black'},})

# average line for January 2020
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=0.125, xref="paper", y0=20.843100, y1=20.843100, yref="y")

# average line for February 2020
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0.125, x1=0.245, xref="paper", y0=16.052474, y1=16.052474, yref="y")

# average line for March 2020
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0.245, x1=0.37, xref="paper", y0=15.651941, y1=15.651941, yref="y")

# average line for April 2020
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0.37, x1=0.495, xref="paper", y0=16.423129, y1=16.423129, yref="y")

# average line for May 2020
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0.495, x1=0.618, xref="paper", y0=17.117166, y1=17.117166, yref="y")

# average line for June 2020
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0.618, x1=0.733, xref="paper", y0=14.968707, y1=14.968707, yref="y")

# average line for July 2020
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0.733, x1=0.8775, xref="paper", y0=16.367532, y1=16.367532, yref="y")

# average line for August 2020
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="red", line_width=3, opacity=1, line_dash="dot",
    x0=0.8775, x1=1, xref="paper", y0=14.720893, y1=14.720893, yref="y")

fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.update_xaxes(range=['2019-12-31', '2020-09-01'])
fig.update_yaxes(range=[0, 50])
fig.update_traces(line_color='black', mode='lines')

fig.show()